# Video Analysis with TwelveLabs

This notebook demonstrates how to analyze videos using the TwelveLabs API, including generating titles, topics, hashtags, summaries, chapters, highlights, and custom open-ended analysis.

## Documentation
- Official API Reference: https://docs.twelvelabs.io/v1.3/sdk-reference/python/analyze-videos

## Overview
TwelveLabs provides multiple methods for analyzing video content:
1. **Gist API** - Generate titles, topics, and hashtags (no custom prompt needed)
2. **Summarize API** - Generate summaries, chapters, and highlights (with optional custom prompts)
3. **Analyze API** - Open-ended analysis with fully customizable prompts and structured output

In [ ]:
# Import required libraries for TwelveLabs API interaction
import uuid  # For generating unique identifiers if needed
from twelvelabs import TwelveLabs  # Main TwelveLabs client class
from twelvelabs.indexes import IndexesCreateRequestModelsItem  # Type hint for index model configuration
from twelvelabs.tasks import TasksRetrieveResponse  # Type hint for task response objects

d:\projects\12labs\new1\.venv\Lib\site-packages\twelvelabs\core\pydantic_utilities.py:13: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.datetime_parse import parse_date as parse_date


In [ ]:
# Load environment variables from .env file
import os
from dotenv import load_dotenv  

# Load environment variables (including API key)
# IMPORTANT: Make sure to actually call load_dotenv() to load variables from .env file
load_dotenv()

# Retrieve the TwelveLabs API key from environment variables
apiKey = os.getenv("TL_API_KEY")

# Verify that the API key was successfully loaded
# This prints True if key exists, False otherwise
print(f"Key loaded: {bool(apiKey)}")

Key loaded: True


Titles, topics, and hashtags
Description: This method analyzes a specific video and generates titles, topics, and hashtags based on its content. It uses predefined formats and doesn’t require a custom prompt, and it’s best for generating immediate and straightforward text representations without specific customization.

In [ ]:
# Import libraries for making HTTP requests and handling JSON data
import requests  # For making HTTP POST requests to the API
import json  # For formatting JSON output

# Specify the video ID you want to analyze
# This should be a video that has been indexed in your TwelveLabs account
video_id = "6953a46fdb0246c06ce30977"

# Generate titles, topics, and hashtags using the Gist API
# POST /gist endpoint analyzes video and returns structured metadata
response = requests.post(
  "https://api.twelvelabs.io/v1.3/gist",  # API endpoint for gist generation
  headers={
    "x-api-key": apiKey  # Your TwelveLabs API key
  },
  json={
    "video_id": video_id,  # ID of the video to analyze
    "types": [
      "title",   # Generate a title for the video
      "topic",   # Generate topics/themes from the video
      "hashtag"  # Generate relevant hashtags
    ]
  },
)

# Print the API response in a formatted JSON structure
# Response includes: id, title, topics (array), hashtags (array), usage (token count)
print(json.dumps(response.json(), indent=2))

{
  "id": "43c357df-3ecc-4ee4-91b8-fe792596f35e",
  "title": "Red Bull Urban DH Race: Aizol India Thrills",
  "topics": [
    "Urban Downhill Mountain Bike Racing"
  ],
  "hashtags": [
    "MountainBiking",
    "UrbanDH",
    "RedBull",
    "MotoGP",
    "AizolIndia"
  ],
  "usage": {
    "output_tokens": 36
  }
}


## Summaries, Chapters, and Highlights (Summarize API)

This method analyzes a video and generates summaries, chapters, or highlights based on its content. Optionally, you can provide a prompt to customize the output format and focus.

**Use Cases:**
- Video summarization for quick understanding
- Chapter generation for long-form content
- Highlight extraction for promotional content
- Customized summaries with specific focus areas

In [ ]:
# Import libraries for making HTTP requests and handling JSON data
import requests  # For making HTTP POST requests to the API
import json  # For formatting JSON output

# Generate summaries, chapters, or highlights using the Summarize API
# POST /summarize endpoint analyzes video and returns structured summaries
response = requests.post(
  "https://api.twelvelabs.io/v1.3/summarize",  # API endpoint for summarization
  headers={
    "x-api-key": apiKey  # Your TwelveLabs API key
  },
  json={
    "video_id": video_id,  # ID of the video to analyze
    "type": "summary",  # Type of analysis: "summary", "chapter", or "highlight"
    "prompt": "Generate a summary of this video for a social media post, up to two sentences.",  # Optional: Custom prompt to guide the analysis
    "temperature": 0.2,  # Controls randomness (0.0 = deterministic, 1.0 = creative)
    "max_tokens": 50  # Maximum number of tokens in the response
  },
)

# Print the API response in a formatted JSON structure
# Response includes: summarize_type, id, summary/chapter/highlight text, finish_reason, usage
print(json.dumps(response.json(), indent=2))

{
  "summarize_type": "summary",
  "id": "435122bb-e88e-4fc1-a257-db31537c22fc",
  "summary": "In Aizol, India, for the first Urban DH race doubling as the Red Bull Cerro Bajo qualifier, Pedro Burns takes us on a high-speed, adrenaline-fueled journey through narrow streets, steep stairs, and slippery ramps, showcasing",
  "finish_reason": "length",
  "usage": {
    "output_tokens": 50
  }
}


## Open-ended Analysis (Analyze API)

This method analyzes a video and generates text based on its content with fully customizable prompts. You can request structured output using JSON schemas and control the generation parameters.

**Use Cases:**
- Custom analysis with specific requirements
- Structured data extraction (tables, lists, objects)
- Content transformation and reformatting
- Detailed analysis with specific focus areas

In [ ]:
# Import libraries for making HTTP requests and handling JSON data
import requests  # For making HTTP POST requests to the API
import json  # For formatting JSON output and parsing streaming responses

# Perform open-ended analysis using the Analyze API
# POST /analyze endpoint provides fully customizable video analysis
response = requests.post(
  "https://api.twelvelabs.io/v1.3/analyze",  # API endpoint for open-ended analysis
  headers={
    "x-api-key": apiKey  # Your TwelveLabs API key
  },
  json={
    "video_id": "6953a46fdb0246c06ce30977",  # ID of the video to analyze
    "prompt": "I want to generate a description for my video with the following format - Title of the video, followed by a summary in 2-3 sentences, highlighting the main topic, key events, and concluding remarks.",  # Custom prompt to guide the analysis
    "temperature": 0.2,  # Controls randomness (0.0 = deterministic, 1.0 = creative)
    "stream": True,  # Enable streaming response for real-time output
    "response_format": {
      "type": "json_schema",  # Request structured JSON output
      "json_schema": {
        "type": "object",
        "properties": {
          "title": {
            "type": "string"  # Title field as string
          },
          "summary": {
            "type": "string"  # Summary field as string
          },
          "keywords": {
            "type": "array",  # Keywords field as array
            "items": {
              "type": "string"  # Array items are strings
            }
          }
        }
      }
    },
    "max_tokens": 100  # Maximum number of tokens in the response
  },
)

# Handle streaming JSON response
# Streaming allows you to see results as they're generated
if response.status_code == 200:
    print("Streaming response:")
    # Iterate through each line in the streaming response
    for line in response.iter_lines():
        if line:  # Skip empty lines
            try:
                # Parse each line as JSON
                data = json.loads(line)
                # Print formatted JSON for each event
                print(json.dumps(data, indent=2))
            except json.JSONDecodeError:
                # If line is not valid JSON, print as text
                print(f"Non-JSON line: {line.decode('utf-8')}")
else:
    # If request failed, print error status and message
    print(f"Error: {response.status_code}")
    print(response.text)

Streaming response:
{
  "event_type": "stream_start",
  "metadata": {
    "generation_id": "bbd23f8a-02af-48ef-bff7-a48dd7f4d3b1"
  }
}
{
  "event_type": "text_generation",
  "text": "{\"keywords\":[\""
}
{
  "event_type": "text_generation",
  "text": "downhill\",\"race\",\""
}
{
  "event_type": "text_generation",
  "text": "mountain\",\"biking"
}
{
  "event_type": "text_generation",
  "text": "\",\"Red\",\"Bull"
}
{
  "event_type": "text_generation",
  "text": "\",\"Alang\",\"Al"
}
{
  "event_type": "text_generation",
  "text": "ang\"],\"summary\":\"The"
}
{
  "event_type": "text_generation",
  "text": " video showcases"
}
{
  "event_type": "text_generation",
  "text": " a high-speed downhill mountain"
}
{
  "event_type": "text_generation",
  "text": " bike race in Aiz"
}
{
  "event_type": "text_generation",
  "text": "ol, India, featuring"
}
{
  "event_type": "text_generation",
  "text": " urban"
}
{
  "event_type": "text_generation",
  "text": " obstacles like stairs and ramps"
}
{
